In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp

from JAxtar.hash import hash_func_builder
from puzzle.lightsout import LightsOut
from heuristic.lightsout_heuristic import LightsOutHeuristic

In [3]:
puzzle = LightsOut(10)

In [4]:
#check batch generation
defualt_state = jax.vmap(puzzle.State.default)(jnp.zeros(10000))
print(defualt_state[0])
states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(0),10))
print(states[0])

┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ ■ ■ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛


In [5]:
heuristic = LightsOutHeuristic(puzzle)
#check heuristic
print("Heuristic")
dist = jax.vmap(heuristic.distance, in_axes=(0, None))(states, defualt_state[0])
print(dist)

Heuristic
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
neighbor_states, cost = jax.vmap(puzzle.get_neighbours, in_axes=(0,0))(states, jnp.ones(10))
print(neighbor_states.shape)
for i in range(100):
    print(neighbor_states[0][i])

shape(board=(10, 100, 100))
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ ■ ■ □ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ ■ ■ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ ■ □ □ □ □ □ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ ■ ■ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ □ □ ■ ┃
┃ ■ □ □ □ □ □ □ □ ■ ■ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ □ □ ■ ┃
┃ ■ ■ □ □ □ 

In [8]:
print("Heuristic")
dist = jax.vmap(heuristic.distance, in_axes=(0, None))(neighbor_states[0], defualt_state[0])
print(dist)

Heuristic
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 4. 2. 2. 2. 2. 2. 2. 2. 2.
 6. 2. 6. 2. 2. 2. 2. 2. 2. 6. 1. 0. 1. 6. 2. 2. 2. 2. 4. 2. 0. 0. 0. 2.
 4. 2. 2. 2.]


In [7]:
# hash func builder 테스트
hash_func = hash_func_builder(puzzle.State)

# 몇 가지 상태 생성
test_states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(42), 5))

# 각 상태에 대해 해시 값 계산
hash_values = jax.vmap(hash_func, in_axes=(0, None))(test_states, 0)

print("테스트 상태:")
for i in range(5):
    print(test_states[i])
    print(hash_values[i])

print("해시 값:")
print(hash_values)

# 해시 값의 고유성 확인
unique_hashes = jnp.unique(hash_values)
print(f"\n고유한 해시 값의 수: {unique_hashes}")
print(f"모든 해시 값이 고유함: {unique_hashes == len(test_states)}")

16
1
4
(100,)
테스트 상태:
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
2319031167
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ ■ ■ ■ □ □ □ □ □ ┃
┃ □ □ □ ■ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
956642521
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ ■ □ □ □ □ □ ┃
┃ □ □ □ ■ ■ ■ □ □ □ □ ┃
┃ □ □ □ □ ■ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━━━━━━━┛
614568297
┏━━━━━━━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ □ ■ ■ ┃
┃ □ □ □ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □